In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentzian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities

data_path = "../tracking/49b_1r/49b_1r_pre_merge/df_linked.parquet"
smooth_verb = False

In [13]:
print("Import data...")
if 1:
    data_path = "../tracking/49b_1r/49b_1r_pre_merge/df_linked.parquet"
    res_path = "./49b_1r/results"
    analysis_data_path = "./49b_1r/analysis_data"
    red_particle_idx = np.array([8]).astype(int)
    fps = 10
    maxLagtime = 100*fps # maximum lagtime to be considered in the analysis, 100 seconds
    v_step = 10
else:
    data_path = "../tracking/25b_25r/df_linked.parquet"
    res_path = "./25b_25r/results"
    analysis_data_path = "./25b_25r/analysis_data"
    red_particle_idx = np.sort(np.array([27, 24, 8, 16, 21, 10, 49, 14, 12, 9, 7, 37, 36, 40, 45, 42, 13, 20, 26, 2, 39, 5, 11, 22, 44])).astype(int)
    fps = 30
    maxLagtime = 100*fps # maximum lagtime to be considered in the analysis, 100 seconds
    v_step = 30

rawTrajs = pd.read_parquet(data_path)
nDrops = int(len(rawTrajs.loc[rawTrajs.frame==0]))
red_mask = np.zeros(nDrops, dtype=bool)
red_mask[red_particle_idx] = True
colors = np.array(['b' for i in range(nDrops)])
colors[red_particle_idx] = 'r'

frames = rawTrajs.frame.unique().astype(int)
nFrames = len(frames)
print(f"Number of Droplets: {nDrops}")
print(f"Number of Frames: {nFrames} at {fps} fps --> {nFrames/fps:.2f} s")

# ANALYSIS PARAMETERS
pxDimension = 1 # has to be defined 

x = np.arange(1, maxLagtime/fps + 1/fps, 1/fps) # range of power law fit

# WINDOWED ANALYSIS PARAMETERS
window = 300*fps # 320 s
stride = 10*fps # 10 s
print("Windowed analysis args:")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"window of {window/fps} s, stride of {stride/fps} s --> {nSteps} steps")
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

Import data...
Number of Droplets: 50
Number of Frames: 30000 at 10 fps --> 3000.00 s
Windowed analysis args:
window of 300.0 s, stride of 10.0 s --> 270 steps


# MSD

In [15]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True
smooth_verb = False

print(f"MSD Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/msd.ipynb

MSD Analysis: show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global IMSD
Global EMSD
Blue Particles: a = 1.178 ± 0.002, K = 61.499 ± 0.431
Red Particle: a = 0.763 ± 0.001, K = 1475.92 ± 8.618
Windowed IMSD


100%|██████████| 270/270 [05:49<00:00,  1.29s/it]
16:24:49 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:24:49 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ././49b_1r/results/mean_squared_displacement/windowed_analysis/IMSD_wind.mp4


Windowed EMSD


100%|██████████| 270/270 [00:02<00:00, 108.76it/s]
16:25:35 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:25:35 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ././49b_1r/results/mean_squared_displacement/windowed_analysis/EMSD_wind.mp4


# VELOCITY DISTRIBUTION

$$ \huge p(v) = \frac{v}{\sigma^2} e^{-\frac{v^2}{2\sigma^2}} $$

In [25]:
show_verb = False
plot_verb = True
animated_plot_verb = True
save_verb = True
smooth_verb = False

print(f"Speed Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_distr.ipynb

Speed Analysis: show_verb = False, animated_plot_verb = True

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 2.842059051868812 ± 0.019462201664093782
Red Particle σ: 8.527733950350948 ± 0.175593143474466

 Windowed speed distribution Analysis


100%|██████████| 270/270 [01:05<00:00,  4.15it/s]
17:41:37 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
17:41:37 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ././49b_1r/results/speed_distribution/speed_raw_wind.mp4


# TURNING ANGLES DISTRIBUTIONS

In [5]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True
smooth_verb = False

# choose the distribution to fit
distribution_str = "lorentzian"

if distribution_str == "gaussian":
    distribution = normal_distr
elif distribution_str == "lorentzian":
    distribution = lorentzian_distr
    distribution_str = "lorentzian"
else:
    raise ValueError("distribution_str must be either 'gaussian' or 'lorentzian'")

print(f"Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/turning_angles.ipynb

Turning Angles Analysis: show_verb = False, animated_plot_verb = True

 Global turning angles analysis
Raw Trajectories
Blue Particles σ: 0.27552311686634195 ± 0.0031321569279026697, μ: -0.0001749427467774336 ± 0.0031312785630483696
Red Particle σ: 0.18034505358162833 ± 0.002232638268296026, μ: 0.007301939331799477 ± 0.0022323670065862856

 Windowed turning angles analysis


17:35:20 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
17:35:20 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ././49b_1r/results//turning_angles/lorentzian/turn_ang_wind_raw.mp4


# VELOCITY AUTOCORRELATION

In [6]:
show_verb = False
run_windowed_analysis = True
plot_verb = True
animated_plot_verb = True
save_verb = True
overwrite = False
smooth_verb = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

Velocity Autocorrelation Analysis: show_verb = False, run_windowed_analysis = True, animated_plot_verb = True
Global Velocity Autocovariance Function
Windowed analysis


100%|██████████| 268/268 [37:32<00:00,  8.40s/it] 
16:27:35 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:27:35 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/velocity_autocovariance/vacf_wind_raw.mp4
16:27:55 [INFO] Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
16:27:55 [INFO] MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 800x500 -pix_fmt rgba -r 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -vcodec libx264 -y ./stardist_results/velocity_autocovariance/vacf_wind_smooth.mp4


# RADIAL DISTRIBUTON FUNCTION

In [4]:
run_analysis_verb = True
show_verb = False
animated_plot_verb = True
save_verb = True
smooth_verb = False

print(f"Radial Distribution Function Analysis: {traj_verb}, run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: stardist, run_analysis_verb = True, show_verb = False, animated_plot_verb = True
RDF - Raw Trajectories
raw


  1%|          | 255/30000 [00:45<1:38:24,  5.04it/s]

KeyboardInterrupt: 

KeyboardInterrupt: 

# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb